In [2]:
import os
import yaml
import json
import torch
import shutil
import numpy as np
import pandas as pd
import ultralytics
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

print("Torch CUDA is available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

Torch CUDA is available: True
GPU name: NVIDIA GeForce RTX 3090


In [3]:
input_dir = 'annotated_outdoor_yolo'
output_dir = 'dataset_yolo'

os.makedirs(os.path.join(output_dir), exist_ok=True)

images_dir = os.path.join(input_dir, 'images')
labels_dir = os.path.join(input_dir, 'labels')

## Setup

In [4]:
# Get the list of image files (assuming all formats are .jpg, change if needed)
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".jpg")])

# Create dataset indices
dataset_size = len(image_files)
indices = list(range(dataset_size))

# Split into train (70%), val (10%), test (20%)
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=199742069)
train_indices, val_indices = train_test_split(train_indices, test_size=0.125, random_state=199742069)

# Store indices in a dictionary
split_data = {
    "train": train_indices,
    "val": val_indices,
    "test": test_indices
}

print(f"Train set size: {len(train_indices)} - {len(train_indices)/len(indices)*100:.2f}%")
print(f"Validation set size: {len(val_indices)} - {len(val_indices)/len(indices)*100:.2f}%")
print(f"Test set size: {len(test_indices)} - {len(test_indices)/len(indices)*100:.2f}%\n")

# Save the split indices as JSON
json_split_path = os.path.join(output_dir, "dataset_split.json")
with open(json_split_path, "w") as f:
    json.dump(split_data, f, indent=4)

print(f"Dataset split saved to {json_split_path}")

Train set size: 217 - 69.77%
Validation set size: 31 - 9.97%
Test set size: 63 - 20.26%

Dataset split saved to dataset_yolo/dataset_split.json


In [5]:
# List and sort both image and label files
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

# Sanity check
assert len(image_files) == len(label_files), "Image and label counts do not match!"

# Define dataset splits
splits = ["train", "val", "test"]

# Create output directories
for split in splits:
    for category in ["images", "labels"]:
        os.makedirs(os.path.join(output_dir, split, category), exist_ok=True)

# Loop over your splits
for split, indices in split_data.items():
    for idx in indices:
        image_name = image_files[idx]
        label_name = label_files[idx]

        image_src = os.path.join(images_dir, image_name)
        label_src = os.path.join(labels_dir, label_name)

        image_dest = os.path.join(output_dir, split, "images", image_name)
        label_dest = os.path.join(output_dir, split, "labels", label_name)

        shutil.copy(image_src, image_dest)
        shutil.copy(label_src, label_dest)

# Print final counts for verification
print("\nCopy complete! Summary:\n")
for split in split_data.keys():
    image_folder = os.path.join(output_dir, split, "images")
    label_folder = os.path.join(output_dir, split, "labels")

    num_images = len([f for f in os.listdir(image_folder) if f.endswith('.jpg')]) if os.path.exists(image_folder) else 0
    num_labels = len([f for f in os.listdir(label_folder) if f.endswith('.txt')]) if os.path.exists(label_folder) else 0

    print(f"Split '{split}': {num_images} images, {num_labels} labels.")


Copy complete! Summary:

Split 'train': 217 images, 217 labels.
Split 'val': 31 images, 31 labels.
Split 'test': 63 images, 63 labels.


In [6]:
# Create a .yaml file for the YOLO trainer configuration
dataset_yaml_path = os.path.join(output_dir, 'dataset.yaml')
classes_path = os.path.join(input_dir, 'classes.txt')

with open(classes_path, 'r') as f:
    names = [line.strip() for line in f if line.strip()]

print(names)

# Construct .yaml file to include dataset paths
data_yaml = {
    "names": names,
    "nc": len(names),
    "train": os.path.abspath(os.path.join(output_dir, "train", "images")),
    "val": os.path.abspath(os.path.join(output_dir, "val", "images")),
    "test": os.path.abspath(os.path.join(output_dir, "test", "images"))
}

# Save to .yaml file
with open(dataset_yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"YAML file saved at {dataset_yaml_path}")


['bus', 'bus_station', 'car', 'crossroad', 'person', 'pothole', 'stairs_down', 'stairs_up']
YAML file saved at dataset_yolo/dataset.yaml


## Model Training

In [29]:
model = YOLO('yolov8n.pt')

In [30]:
# Define parameters for training the YOLO model
yolo_params = {
    'image_size' : 640,
    'batch_size' : 16,
    'epochs' : 200
}

dataset_yaml_path = os.path.join(output_dir, 'dataset.yaml')

# Train the model
results = model.train(
            data=dataset_yaml_path,
            imgsz=yolo_params['image_size'],
            epochs=yolo_params['epochs'],
            batch=yolo_params['batch_size'],
            name=f'yolov8n_outdoor_train',
            project=os.path.join(output_dir, 'runs'),
            device=0,
            patience=0 
        )

New https://pypi.org/project/ultralytics/8.3.111 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_yolo/dataset.yaml, epochs=200, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=dataset_yolo/runs, name=yolov8n_outdoor_train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

train: Scanning /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/labels... 280 images, 0 backgrounds, 0 corrupt: 100%|██████████| 280/280 [00:00<00:00, 2550.09it/s]

train: New cache created: /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/labels.cache



val: Scanning /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/1552f1ae-urban_scenes_58.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/15e532ad-urban_scenes_25.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/16b51f6d-urban_scenes_5.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/22ae1e24-urban_scenes_100.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/4b237d1d-urban_scenes_61.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/55dfe649-urban_scenes_102.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_M

...

    200/200      2.62G     0.7277      0.506     0.9104         22        640: 100%|██████████| 18/18 [00:01<00:00, 11.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 12.78it/s]

                   all         31        191       0.67      0.613      0.615      0.349



200 epochs completed in 0.131 hours.
Optimizer stripped from dataset_yolo/runs/yolov8n_outdoor_train6/weights/last.pt, 6.3MB
Optimizer stripped from dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.pt, 6.3MB

Validating dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.22it/s]


                   all         31        191      0.747      0.589      0.645      0.418
                   bus          4          9      0.729      0.778      0.864      0.638
           bus_station          6          8      0.864      0.625       0.63      0.522
                   car         18        100      0.733      0.522      0.558      0.326
             crossroad          6         14      0.539      0.429      0.469      0.293
                person         10         25      0.842      0.427      0.527      0.185
               pothole         10         22      0.414      0.417      0.362      0.236
           stairs_down          4          6      0.857      0.667      0.817      0.592
             stairs_up          6          7          1      0.847      0.937      0.551
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to dataset_yolo/runs/yolov8n_outdoor_train6


## Test

In [31]:
best_model_path = os.path.join(output_dir,'runs','yolov8n_outdoor_train6','weights','best.pt')
best_model = YOLO(best_model_path)

In [33]:
test_results = best_model.val(data=dataset_yaml_path, split='val', name=f'val')
print(f"mAP@50 test results: {test_results.box.map50}")

Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)


val: Scanning /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/1552f1ae-urban_scenes_58.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/15e532ad-urban_scenes_25.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/16b51f6d-urban_scenes_5.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/22ae1e24-urban_scenes_100.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/4b237d1d-urban_scenes_61.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/55dfe649-urban_scenes_102.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_M


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]


                   all         31        191      0.733      0.628      0.653      0.405
                   bus          4          9      0.725      0.778      0.869      0.607
           bus_station          6          8      0.873      0.625       0.63       0.49
                   car         18        100        0.7       0.54      0.569      0.326
             crossroad          6         14      0.523        0.5      0.479      0.271
                person         10         25       0.81       0.48      0.528      0.187
               pothole         10         22      0.432       0.45      0.348      0.232
           stairs_down          4          6          1      0.796      0.846      0.556
             stairs_up          6          7      0.797      0.857      0.953       0.57
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val
mAP@50 test results: 0.6525516731717436


## Export as ONNX

In [34]:
# Export trained model to .onnx to later convert to .blob using BlobConverter
#best_model.export(format='onnx', dynamic=True, simplify=True)
best_model.export(format='onnx', dynamic=False, simplify=True, imgsz=(640, 640))

Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CPU (AMD Ryzen 9 5900X 12-Core Processor)

PyTorch: starting from 'dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 0.5s, saved as 'dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.onnx' (11.7 MB)

Export complete (0.6s)
Results saved to /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/runs/yolov8n_outdoor_train6/weights
Predict:         yolo predict task=detect model=dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.onnx imgsz=640 data=dataset_yolo/dataset.yaml  
Visualize:       https://netron.app


'dataset_yolo/runs/yolov8n_outdoor_train6/weights/best.onnx'